# Lesson 4 - Understanding User Intent for Global Position Management Operations

In this lesson, you will build the first agent of your multi-agent system, which will be the user intent agent.

You'll learn:

- how to give an agent a clear task
- how to define tools that are appropriate for the task
- how use state to save important information

Along the way, you will get experience with basic human-in-the-loop interaction


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>To access the helper.py and neo4j_for_adk.py files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.
</div>

## 4.1. Agent Details

<img src="images/entire_solution.png" width="500">

The user intent agent is a goal-oriented, conversational agent that helps the Global Position Management Operations team ideate on the kind of graph to build for capital markets operations.
- Input: nothing
- Output: `approved_user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.
- Tools: `set_perceived_user_goal`, `approve_perceived_user_goal`

**Note**: Each lesson will focus on one part of the multi-agent system (as shown in the above image). The end-to-end solution will be available in this [repo](https://github.com/neo4j-contrib/agentic-kg). If the repo is empty, it means that the project is still in progress.

## 4.2. Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [6]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from networkx_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [12]:
# --- Define Model Constants for easier use ---

# Option 1: OpenAI (Requires paid API key)
MODEL_GPT_4O = "openai/gpt-4o"

# Option 2: Groq (FREE - Fast & Generous free tier)
# Get free API key from: https://console.groq.com
# Set in .env: GROQ_API_KEY=your_key_here
MODEL_GROQ = "groq/llama-3.1-70b-versatile"

# Option 3: Google Gemini (FREE tier available)
# Get free API key from: https://makersuite.google.com/app/apikey
# Set in .env: GEMINI_API_KEY=your_key_here
MODEL_GEMINI = "gemini/gemini-1.5-flash"

# Option 4: Ollama (FREE - Runs locally, no API key needed)
# Install Ollama from: https://ollama.com
# Run: ollama pull llama3.2
MODEL_OLLAMA = "ollama/llama3.2"

# Option 5: Mock/Test Mode (No API needed - for testing workflow)
# This uses a simple mock that returns test responses
USE_MOCK_MODE = True  # Set to True if you can't connect to any API

# Choose your model here:
SELECTED_MODEL = MODEL_OLLAMA  # Changed to Ollama for offline use

if USE_MOCK_MODE:
    print("🧪 MOCK MODE: Using test responses (no real LLM)")
    # Create a simple mock LLM for testing
    class MockLLM:
        def __init__(self):
            self.model = "mock/test"
            self.llm_client = self
        
        def completion(self, **kwargs):
            return {"choices": [{"message": {"content": "Mock response: Yes, I'm ready!"}}]}
    
    llm = MockLLM()
else:
    llm = LiteLlm(model=SELECTED_MODEL)

# Test LLM with a direct call
try:
    print(f"Testing model: {SELECTED_MODEL if not USE_MOCK_MODE else 'MOCK MODE'}")
    result = llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[])
    print(result)
    print(f"\n✅ {SELECTED_MODEL if not USE_MOCK_MODE else 'Mock LLM'} is ready!")
except Exception as e:
    print(f"\n❌ Error connecting to {SELECTED_MODEL}")
    print(f"Error: {str(e)}")
    print("\n💡 Solutions:")
    print("1. If offline: Install Ollama from https://ollama.com and run 'ollama pull llama3.2'")
    print("2. If you have internet: Get free API key from https://console.groq.com")
    print("3. For testing only: Set USE_MOCK_MODE = True above (line 25)")
    raise

🧪 MOCK MODE: Using test responses (no real LLM)
Testing model: MOCK MODE
{'choices': [{'message': {'content': "Mock response: Yes, I'm ready!"}}]}

✅ Mock LLM is ready!


## 🔧 Troubleshooting Connection Issues

If you see **"getaddrinfo failed"** or **"connection error"**, you have 3 options:

### Option 1: Ollama (Local - No Internet) ⭐ Recommended for Offline
1. Download from: https://ollama.com/download
2. Install and run: `ollama pull llama3.2`
3. Set `SELECTED_MODEL = MODEL_OLLAMA` in cell below
4. No API key needed!

### Option 2: Groq (Cloud - Requires Internet)
1. Visit: https://console.groq.com/keys
2. Get free API key
3. Add to `.env` file: `GROQ_API_KEY=your_key`
4. Set `SELECTED_MODEL = MODEL_GROQ`

### Option 3: Mock Mode (Testing Only)
- Set `USE_MOCK_MODE = True` in cell below
- Workflow will work but won't have real AI responses

## 4.3. Define the User Intent Agent

### 4.3.1 Agent Instructions

You will define the instructions for the agent one part at a time, then combine them into a final set of instructions.

In [13]:
# define the role and goal for the user intent agent
agent_role_and_goal = """
    You are an expert at knowledge graph use cases for capital markets and investment banking operations. 
    Your primary goal is to help the Global Position Management Operations team come up with a knowledge graph use case 
    that supports their operational, risk management, and regulatory compliance needs.
"""

In [14]:
# give the agent some hints about what to say
agent_conversational_hints = """
    If the user is unsure what to do, make some suggestions based on capital markets use cases like:
    - position aggregation network showing relationships between trades, positions, accounts, books, and legal entities
    - counterparty risk network tracking exposures, collateral, and credit relationships across trading partners
    - securities master data graph connecting instruments, issuers, exchanges, and reference data sources
    - reconciliation and break management system tracking discrepancies across internal systems and external custodians
    - regulatory reporting lineage showing data flows from source systems through transformations to regulatory submissions
    - collateral optimization network modeling eligible securities, haircuts, and collateral agreements
    - corporate actions impact analysis tracking entitlements, positions, and downstream effects
    - settlement and clearing workflow with SSIs (Standard Settlement Instructions), nostro accounts, and CSDs (Central Securities Depositories)
"""

In [15]:
# describe what the output should look like
agent_output_definition = """
    A user goal has two components:
    - kind_of_graph: at most 3-4 words describing the graph, for example "position hierarchy network" or "counterparty risk graph"
    - description: a few sentences about the intention of the graph, for example "A comprehensive view of position aggregation across trading books, legal entities, and regulatory reporting hierarchies." or "Real-time counterparty exposure tracking with collateral and netting set relationships for credit risk management."
"""

In [16]:
# specify the steps the agent should follow
agent_chain_of_thought_directions = """
    Think carefully and collaborate with the user:
    1. Understand the user's goal, which is a kind_of_graph with description
    2. Ask clarifying questions as needed
    3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
    4. Present the perceived user goal to the user for confirmation
    5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""


In [17]:
# combine all the instruction components into one complete instruction...
complete_agent_instruction = f"""
{agent_role_and_goal}
{agent_conversational_hints}
{agent_output_definition}
{agent_chain_of_thought_directions}
"""

print(complete_agent_instruction)




    You are an expert at knowledge graph use cases for capital markets and investment banking operations. 
    Your primary goal is to help the Global Position Management Operations team come up with a knowledge graph use case 
    that supports their operational, risk management, and regulatory compliance needs.


    If the user is unsure what to do, make some suggestions based on capital markets use cases like:
    - position aggregation network showing relationships between trades, positions, accounts, books, and legal entities
    - counterparty risk network tracking exposures, collateral, and credit relationships across trading partners
    - securities master data graph connecting instruments, issuers, exchanges, and reference data sources
    - reconciliation and break management system tracking discrepancies across internal systems and external custodians
    - regulatory reporting lineage showing data flows from source systems through transformations to regulatory submissio

### 4.3.2 Tool Definition

Using tools to define the user goal helps the agent focus on the requirements. 

Rather than open-ended prose, the user goal is defined with specifc arguments passed to a tool.

In [18]:
# Tool: Set Perceived User Goal
# to encourage collaboration with the user, the first tool only sets the perceived user goal

PERCEIVED_USER_GOAL = "perceived_user_goal"

def set_perceived_user_goal(kind_of_graph: str, graph_description:str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-4 word definition of the kind of graph, for example "position management hierarchy" or "counterparty exposure network"
        graph_description: a single paragraph description of the graph, summarizing the user's intent and operational objectives
    """
    user_goal_data = {"kind_of_graph": kind_of_graph, "graph_description": graph_description}
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)

In [19]:
# Tool: Approve the perceived user goal
# approval from the user should trigger a call to this tool

APPROVED_USER_GOAL = "approved_user_goal"

def approve_perceived_user_goal(tool_context: ToolContext):
    """Upon approval from user, will record the perceived user goal as the approved user goal.
    
    Only call this tool if the user has explicitly approved the perceived user goal.
    """
    # Trust, but verify. 
    # Require that the perceived goal was set before approving it. 
    # Notice the tool error helps the agent take
    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Set perceived user goal first, or ask clarifying questions if you are unsure.")
    
    tool_context.state[APPROVED_USER_GOAL] = tool_context.state[PERCEIVED_USER_GOAL]

    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])


In [20]:
# add the tools to a list
user_intent_agent_tools = [set_perceived_user_goal, approve_perceived_user_goal]

### 4.3.3 Agent Definition

In [21]:
# Finally, construct the agent

user_intent_agent = Agent(
    name="user_intent_agent_v1", # a unique, versioned name
    model=llm, # defined earlier in a variable
    description="Helps the user ideate on a knowledge graph use case.", # used for delegation
    instruction=complete_agent_instruction, # the complete instructions you composed earlier
    tools=user_intent_agent_tools, # the list of tools
)

print(f"Agent '{user_intent_agent.name}' created.")

ValidationError: 2 validation errors for LlmAgent
model.str
  Input should be a valid string [type=string_type, input_value=<__main__.MockLLM object at 0x000001DD1AC6F680>, input_type=MockLLM]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
model.BaseLlm
  Input should be a valid dictionary or instance of BaseLlm [type=model_type, input_value=<__main__.MockLLM object at 0x000001DD1AC6F680>, input_type=MockLLM]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type

## 4.4. Interact with the Agent

In [ ]:
# use a helper to create an agent execution environment
from helper import make_agent_caller

# NOTE: if re-running the session, come back here to re-initialize the agent
user_intent_caller = await make_agent_caller(user_intent_agent)

In [ ]:
# Run the Initial Conversation

session_start = await user_intent_caller.get_session()
print(f"Session Start: {session_start.state}") # expect this to be empty

# We need an async function to await for each conversation
async def run_conversation():
    # start things off by describing your goal
    await user_intent_caller.call("""I'd like a position management hierarchy graph that tracks the complete lineage from individual trades 
    up through positions, books, desks, legal entities, and regulatory reporting structures. This should support position reconciliation, 
    P&L attribution, and regulatory reporting (e.g., CCAR, Volcker, FRTB).""") 

    if PERCEIVED_USER_GOAL not in session_start.state:
        # the LLM may have asked a clarifying question. offer some more details
        await user_intent_caller.call("""We need to handle complex scenarios like inter-desk transfers, novations, and real-time 
        position updates across multiple asset classes including equities, fixed income, derivatives, and FX.""")        

    # Optimistically presume approval.
    await user_intent_caller.call("Approve that goal.", True)

await run_conversation()

session_end = await user_intent_caller.get_session()

Take a close look at the session state.

- session state starts empty, as you'd expect since we did not initialize it
- after the conversation, there are two values in session state: `perceived_user_goal` and `approved_user_goal`
- this duplication is intentional, separating "working memory" from work specification
- subsequent steps in the workflow should only use approved work specifications
- in production, work specifications should be persisted to enable tracing and reproducibility


## 4.5. Optional - Sequence diagram illustrating the workflow of  "User Intent Agent"  

<img src="images/user_intent_diag.png"  width=600> 